# Python - Testat 2

Initialisierung:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline
import matplotlib.colors as colors

# make a colormap that has land and ocean clearly delineated and of the
# same length (256 + 256)
colors_undersea = plt.cm.terrain(np.linspace(0, 0.17, 256))
colors_land = plt.cm.terrain(np.linspace(0.25, 1, 256))
all_colors = np.vstack([colors_undersea, colors_land])
terrain_map = colors.LinearSegmentedColormap.from_list(
    'terrain_map', all_colors)

class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=True):
        self.vcenter = vcenter
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

# Contourf-Parameter
c_levels = np.linspace(-800, 1700, 26)
midnorm = MidpointNormalize(vmin=-800, vcenter=0, vmax=1700)

## TopoMap-Instanz erstellen

In [ ]:
from testat2_muloe import TopoMap

In [ ]:
tmap = TopoMap('map_data.txt')

In [ ]:
tmap.X.shape, tmap.Y.shape, tmap.Z.shape

In [ ]:
if tmap.X is not tmap.X and tmap.Y is not tmap.Y and tmap.Z is not tmap.Z:
    print('richtig: X/Y/Z sind Kopien der internen Daten.')
else:
    print('falsch: Referenzen auf interne Daten.')

## highest_point(self)

In [ ]:
P_H = tmap.highest_point()
P_H

## lowest_point(self)

In [ ]:
P_L = tmap.lowest_point()
P_L

## elevation(self, position)

In [ ]:
tmap.elevation([1.1e3, -2.2e3])

## elevation_profile(self, path)

In [ ]:
# Pfad A-B-C definieren
PA = [-9e3, -6e3]
PB = [-8e3, -1.5e3]
PC = [6e3, -2.5e3]
pfad_ABC = np.c_[np.r_[np.linspace(PA[0], PB[0], 500),
                       np.linspace(PB[0], PC[0], 500)],
                 np.r_[np.linspace(PA[1], PB[1], 500),
                       np.linspace(PB[1], PC[1], 500)]]

In [ ]:
dn, zn = tmap.elevation_profile(pfad_ABC)

In [ ]:
fig, ax = plt.subplots()
ax.plot(dn, zn)
ax.set_xlabel('2D-Distanz (m)')
ax.set_ylabel('Höhe (m ü. M.)')
ax.grid(True)
fig.tight_layout()

## distance_travelled(self, path)

In [ ]:
d, hm = tmap.distance_travelled(pfad_ABC)

In [ ]:
print('Weglänge = {:.0f}m, Höhenmeter = {:.0f}m'. format(d, hm))

## fall_line(self, start, descent=True, gamma=50, max_iter=10000, precision=0.1)

In [ ]:
# Startpunkte definieren
P1 = [6200, -5100]
P2 = [-2500, -3000]

In [ ]:
tmap.fall_line(P1)

In [ ]:
tmap.fall_line(P2, descent=False)

## aligned_map(self, p, q)

In [ ]:
# Referenzpunkte
P = [5165, -6601]
Q = [-7052, -4115]

In [ ]:
X_new, Y_new, Z_new, A_rot = tmap.aligned_map(P, Q)
print('shapes:', X_new.shape, Y_new.shape, Z_new.shape)
print('A =\n', A_rot, sep='')

In [ ]:
Q_new = [0, 1.24673688e+04]

In [ ]:
fig, ax = plt.subplots()
ax.contourf(
        X_new,
        Y_new,
        Z_new,
        c_levels,
        norm=midnorm,
        cmap=terrain_map,
)
ax.axis('image')
ax.grid(True)
ax.plot(0, 0, 'b^', label='P')
ax.plot(Q_new[0], Q_new[1], 'ro', label='Q')
ax.set_xlabel(r'$\tilde{x}$ (m)')
ax.set_ylabel(r'$\tilde{y}$ (m)')
ax.legend()
fig.tight_layout()